# A Full Business Solution for a Investment Management Company

### BUSINESS CHALLENGE:

Create a product that builds a 1 Page Pitch for an investment company to convince them to purchase a stock and include it in their portfolio.


In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
I will use "one shot prompting" in which I provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a 1 page pitch about the company for investment managers who want to buy the stock, \
such as links to an About page, or a Company page, or Investor Relations pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "investor relations page": "url": "https://another.full.url/investor-relations"}
    ]
}
"""

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a pitch 1 pager about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

## Second Step: Make the Pitch Deck!

Assemble all the details into another prompt to GPT4-o

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [8]:
system_prompt = "You are an investment research analyst that analyzes the contents of several relevant pages from a company website \
and creates a short 1 page pitch about the company for prospective investment managers as to why to purchase the stock. Respond in markdown.\
Include details of company products, services, investor relations, growth, and other details if you have the information that will help the analyst give a buy recommendation."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [9]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short 1 page pitch about the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20000] # Truncate if more than 5,000 characters
    return user_prompt

In [10]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

## Finally - A Minor Improvement

With a small adjustment, I can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [11]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [12]:
stream_brochure("Blackrock", "https://blackrock.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.blackrock.com/us/individual/about-us/about-blackrock'}, {'type': 'corporate page', 'url': 'https://www.blackrock.com/corporate'}, {'type': 'investor relations page', 'url': 'https://ir.blackrock.com/home/default.aspx'}, {'type': 'investment stewardship page', 'url': 'https://www.blackrock.com/corporate/about-us/investment-stewardship'}, {'type': 'sustainability page', 'url': 'https://www.blackrock.com/corporate/sustainability/our-approach-to-sustainability'}, {'type': 'newsroom page', 'url': 'https://www.blackrock.com/corporate/newsroom'}]}


# Investment Pitch: BlackRock (BLK)

## Overview
BlackRock, Inc. (NYSE: BLK) is a premier global investment manager and fiduciary with over $9 trillion in assets under management (AUM), spanning diverse investment strategies and geographical markets. Established in 1988, the firm's mission is to help more people experience financial well-being by delivering innovative investment solutions and services.

## Key Products & Services
BlackRock’s offerings cater to various types of clients, including individual investors, advisors, institutions, and government entities. Their key product lines include:

- **Mutual Funds:** A wide range of actively managed and index funds covering various asset classes.
- **iShares ETFs:** Market-leading exchange-traded funds (ETFs) that provide exposure to equities, fixed income, commodities, and more.
- **Aladdin Platform:** Advanced investment management and technology platform providing risk analytics, portfolio management, and financial stewardship.
- **Retirement Solutions:** Target date funds like LifePath, which help investors plan for retirement by adjusting exposure based on the target retirement date.

## Growth Potential
1. **Market Leadership:** As one of the largest asset managers, BlackRock's scale enables it to leverage efficiencies and maintain competitive pricing. The growth of passive investment strategies further solidifies its position in the ETF market.
2. **Digital Assets:** BlackRock is proactively exploring digital asset investments, including cryptocurrencies and blockchain-related opportunities, aligning with the evolving preferences of younger investors.
3. **Sustainability Focus:** The firm's commitment to sustainability integrates Environmental, Social, and Governance (ESG) factors into its investment processes, attracting socially responsible investors and staying ahead of regulatory demands.

## Investor Relations
BlackRock maintains a robust investor relations framework, providing detailed financial disclosures, yearly earnings discussions, and transparency in corporate governance. Larry Fink, CEO, emphasizes the importance of responsible investing and the need for adaptability in the ever-changing market landscape. The company is committed to delivering shareholder returns through dividends and stock buybacks, reinforcing investor confidence.

## Recent Developments & Insights
- Recent commentary from BlackRock highlights the importance of smart portfolio allocations amidst economic uncertainties, advocating for a balanced approach between stocks, bonds, commodities, and alternative investments.
- The firm continually enhances its digital tools, such as the LifePath cash management solutions and Portfolio Selector Tool, aimed at improving user engagement and investment outcomes for clients.

## Conclusion: Buy Recommendation
With its strong market position, diversified product offerings, commitment to innovation, and focus on sustainability, BlackRock is well-positioned for continued growth and resilience in changing markets. Investors seeking to capitalize on long-term financial trends and robust returns should consider adding BlackRock to their portfolios. Given these factors, we recommend a **Buy** rating for BlackRock's stock.